In [10]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
%matplotlib inline
from __future__ import print_function
import collections
import math
import numpy as np
import os
import random
import tensorflow as tf
import zipfile
import pandas as pd
from matplotlib import pylab
from six.moves import range
from six.moves.urllib.request import urlretrieve
from sklearn.manifold import TSNE
from sklearn.cross_validation import train_test_split
import time

In [11]:
# download data
url = 'https://raw.githubusercontent.com/chiphuyen/tf-stanford-tutorials/master/data/'

def maybe_download(filename):
    if not os.path.exists(filename):
        filename, _ = urlretrieve(url + filename, filename)
    statinfo = os.stat(filename)
    return filename

filename = maybe_download('heart.csv')

In [45]:
# Pre-treatment
raw_data = pd.read_csv('heart.csv')

label = raw_data['chd']

data = raw_data.loc[:,raw_data.columns[:9]].replace(['Present', 'Absent'],[0, 1])
data = data.apply(lambda x: (x - np.mean(x)) / (np.max(x) - np.min(x)), axis=0)
X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.10, random_state=0)

print(X_train.shape)
print(X_test.shape)

X_train.describe()


(415, 9)
(47, 9)


,sbp,tobacco,ldl,adiposity,famhist,typea,obesity,alcohol,age
count,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000
mean,-0.007716,0.001157,0.001615,0.000246,0.003536,0.006372,0.001294,-0.006021,-0.005245
std,0.170554,0.150793,0.145453,0.218390,0.492798,0.148035,0.132303,0.158801,0.297527
min,-0.319033,-0.116527,-0.262044,-0.522146,-0.584416,-0.616983,-0.355838,-0.115799,-0.567674
25%,-0.122452,-0.114444,-0.100023,-0.158790,-0.584416,-0.078521,-0.094859,-0.113285,-0.241143
50%,-0.036982,-0.052425,-0.025807,0.019392,0.415584,0.013786,-0.007657,-0.071366,0.024163
75%,0.074130,0.056550,0.073845,0.165126,0.415584,0.106094,0.081113,0.039341,0.248653
max,0.680967,0.883473,0.737956,0.477854,0.415584,0.383017,0.644162,0.871293,0.432326


In [43]:
# Define paramaters for the model
learning_rate = 0.01
batch_size = 5
n_epochs = 10

In [71]:
X = tf.placeholder(dtype = np.float32, shape = [9, None], name='X')
Y = tf.placeholder(dtype = np.float32, shape = [1, None], name='Y')

tf.set_random_seed(1)

W1 = tf.Variable(tf.random_uniform([12, 9]))
b1 = tf.Variable(tf.random_uniform([12, 1]))
W2 = tf.Variable(tf.random_uniform([6, 12]))
b2 = tf.Variable(tf.random_uniform([6, 1]))
W3 = tf.Variable(tf.random_uniform([1, 6]))
b3 = tf.Variable(tf.random_uniform([1, 1]))

# W = tf.Variable(tf.random_uniform([784, 10]))

Z1 = tf.add(tf.matmul(W1, X), b1)                      # Z1 = np.dot(W1, X) + b1
A1 = tf.nn.relu(Z1)                                    # A1 = relu(Z1)
Z2 = tf.add(tf.matmul(W2, A1), b2)                     # Z2 = np.dot(W2, a1) + b2
A2 = tf.nn.relu(Z2)                                    # A2 = relu(Z2)
Z3 = tf.add(tf.matmul(W3, A2), b3)                     # Z3 = np.dot(W3,Z2) + b3

logits = Z3

logits = tf.transpose(Z3)
labels = tf.transpose(Y)

entropy = tf.nn.softmax_cross_entropy_with_logits_v2(labels = labels, logits = logits)

loss = tf.reduce_mean(entropy)

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

# with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())	
#     _, loss_test = sess.run([optimizer, loss], feed_dict={X: X_train.values.T, Y: y_train.values.reshape(1, -1)})
#     print(loss_test)

with tf.Session() as sess:
    start_time = time.time()
    sess.run(tf.global_variables_initializer())	
    n_batches = int(X_train.values.shape[0]/batch_size)
    for i in range(n_epochs): 
        total_loss = 0

        for index in range(n_batches):
            X_batch = X_train[index*batch_size:(index+1)*batch_size].values
            Y_batch = y_train[index*batch_size:(index+1)*batch_size].values
            _, loss_batch = sess.run([optimizer, loss], feed_dict={X: X_batch.T, Y: Y_batch.reshape(1, -1)})
            total_loss += loss_batch
        print('Average loss epoch :{0}'.format(total_loss/n_batches))

    print('Total time: {0} seconds'.format(time.time() - start_time))

    print('Optimization Finished!')

    # test the model
    n_batches = int(X_test.shape[0]/batch_size)

    total_correct_preds = 0
    for index in range(n_batches):
        X_batch = X_test[index*batch_size:(index+1)*batch_size].values
        Y_batch = y_test[index*batch_size:(index+1)*batch_size].values
        print(Y_batch)
        _, loss_batch, logits_batch = sess.run([optimizer, loss, logits], feed_dict={X: X_batch.T, Y: Y_batch.reshape(1, -1)}) 
        preds = tf.nn.softmax(logits_batch)
        correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(Y_batch, 1))
        result = sess.run(correct_preds)
        print(result)

        
        accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32))
        print(accuracy)
        total_correct_preds += sess.run(accuracy)

    print('Accuracy:',format(total_correct_preds/n_batches/batch_size))

Average loss epoch :0.0
Average loss epoch :0.0
Average loss epoch :0.0
Average loss epoch :0.0
Average loss epoch :0.0
Average loss epoch :0.0
Average loss epoch :0.0
Average loss epoch :0.0
Average loss epoch :0.0
Average loss epoch :0.0
Total time: 1.560009241104126 seconds
Optimization Finished!
[1 0 1 1 0]


InvalidArgumentError: Expected dimension in the range [-1, 1), but got 1
	 [[Node: ArgMax_15 = ArgMax[T=DT_INT64, Tidx=DT_INT32, output_type=DT_INT64, _device="/job:localhost/replica:0/task:0/device:CPU:0"](ArgMax_15/input, ArgMax_15/dimension)]]

Caused by op 'ArgMax_15', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python3.5/dist-packages/ipykernel/kernelapp.py", line 478, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.5/dist-packages/zmq/eventloop/ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
  File "/usr/local/lib/python3.5/dist-

In [65]:
a = np.array([1 ,0 ,1 ,1 ,0])
print(a)
print(a.reshape(-1, 1))

[1 0 1 1 0]
[[1]
 [0]
 [1]
 [1]
 [0]]
